In [ ]:
# Code based off the following tutorial: 
# https://charlieoneill.medium.com/predicting-the-price-of-bitcoin-with-multivariate-pytorch-lstms-695bc294130

# Introduction

This is a minimum viable implementation of a learning-based clock bias prediction model. Ideally this model would run locally in a system with limited GPS accessiblity. We have historical IGS data available. 

Currently in progress is trying the model with many ephemeris features vs just SV name, and past clock bias values reported by IGS. This model is beneficial because IGS normally may take ~17 days to push out very exact clock bias predictions. 

The objective is to build an AR LSTM with two features: the SV name and its own previous values (i.e. from the previous 20 days to predict the target variable (Final Clock Bias in seconds) for 20 days in the future).

# Data Preprocessing

This data is readily available from CDDIS and IGS in the form of RNX files for the satellite ephemeris data and CLK files for the final clock bias that has been processed by IGS and produced after ~17 days. However, if only the AR model is being used then only CLK file data is necessary.

Now, the original tutorial is based off financial data, and this has been completed in a different file.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np 
import torch 
import torch.nn as nn
import warnings
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
historical_window = 5 # [minutes]
forward_window = 50 # [minutes]

In [ ]:
filename = "5_resampled_1min_interval.csv"

In [ ]:
df = pd.read_csv(filename, index_col = 'epoch', parse_dates=True)
df = df[["SV Name", "Clock Bias (seconds)", "SV Health", "File"]]
# df = df[["SV Name", "Clock Bias (seconds)", "Clock Bias", "Clock Drift", "Clock Drift Rate", "SV Health", "File"]]
df.head(5)

In [ ]:
df.describe().T

In [ ]:
plt.plot(df[(df["SV Name"] == 1)]["Clock Bias (seconds)"])
plt.xlabel("Time")
plt.ylabel("Clock Bias (seconds)")
plt.xticks(rotation=90)
plt.title("Clock Bias for SV1 over time")
plt.savefig("sv1_clock_bias_plot.png", dpi = 250)
plt.show(); 

In [ ]:
X, y = df.drop(columns=['File']), df['Clock Bias (seconds)'] # in this case, it would be useful to have col names without spaces 
X.shape, y.shape

In [ ]:
mm = MinMaxScaler()
ss = StandardScaler()

X_trans = ss.fit_transform(X)
# y_trans = mm.fit_transform(y.reshape(-1,1)) # reshape has been deprecated https://stackoverflow.com/questions/53723928/attributeerror-series-object-has-no-attribute-reshape 
y_trans = mm.fit_transform(y.values.reshape(-1,1))

In [ ]:
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(input_sequences)):
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        # out_end_ix = end_ix + n_steps_out - 1
        out_end_ix = end_ix + n_steps_out

        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break

        # gather input and output of the pattern 
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix:out_end_ix, -1]
        # seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)
    return np.array(X), np.array(y)

X_ss, y_mm = split_sequences(X_trans, y_trans, historical_window, forward_window)
print(X_ss.shape, y_mm.shape)


In [ ]:
def add_regression_noise(y, std=0.1): 
    noise = np.random.normal(0, std, size=y.shape)
    return y + noise

Let's check if the first sample in <code>y_mm</code> starts at the 100th sample in the original <code>y</code> vector

In [ ]:
assert y_mm[0].all() == y_trans[99:149].squeeze(1).all()

In [ ]:
y_mm[0]

In [ ]:
y_trans[99:149].squeeze(1)

We want to predict the data several months into the future, therefore we can use a training data size of 95% with 5% left for the remaining data for prediction. This gives us a training set of 2763 day (or 7.5 years) and predicting 145 days into the future (5 months). 

In [ ]:
total_samples = len(X)
train_test_cutoff = round(0.10 * total_samples)

In [ ]:
X_train = X_ss[:-train_test_cutoff]
X_test = X_ss[-train_test_cutoff:]

In [ ]:
y_train = y_mm[:-train_test_cutoff]
y_train = add_regression_noise(y_train)
y_test = y_mm[-train_test_cutoff:]

In [ ]:
train_test_cutoff

In [ ]:
print("Training Shape:", X_train.shape, y_train.shape)
print("Testing Shape:", X_test.shape, y_test.shape)

Now we convert our data into tensors by calling <code>torch.tensor()</code> on our object, and setting the property <code>requires_grad = True</code>. Some old PyTorch tutorials might indicate that we need to apply <code>Variable</code> on here but this is deprecated. Now the input tensor to be forward propagated has to be can faciliate automatic back propagation through <code> backward()</code> without being wrapped.

In [ ]:
# convert to pytorch tensors 
X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)

y_train_tensors = torch.Tensor(y_train)
y_test_tensors = torch.Tensor(y_test)

Looking through the documentation for a multi-layer <code>torch.nn.LSTM</code> shows that the input shape depends on whether the parameter <code>batch_first</code> is true. Since we are accustomed to having the first dimension be the batch, we can set <code>batch_first</code> to true. The size of the input is then <code>(N, L, H_in)</code> where

- N is the batch size 
- L is the sequence length
- H_in is the input size (number of features)

In other words, we wantt eh dimensions to be the rows of the dataframe in the first dimension. Followed by the length of the dataframe in the next dimension to represent the length of the input sequence, and finally the feature of which we have 4 in the final dimension. 

To reshape the tensors into our required shape we can use <code>torch.reshape</code>. This takes as arguments the tensors we are reshaping, and then a tuple of the shape we need to reshape to. For the rows of the dataframe, we can simply look at the shape of the first dimension <code>X_train_tensors.shape[0]</code>. Since we are applying an LSTM, we call tha tthe sequence length we feed in is simply 1 because the whole point of the LSTM is not feeding in tons of data at each because the emory is handled by the inner working of the LSTM cell. 

The last dimension is the number of features which is stored in the <code>X_train_tensors.shape[1]</code>. 

In [ ]:
# reshape to rows, timestamps, features 

X_train_tensors_final = torch.reshape(X_train_tensors, 
                                      (X_train_tensors.shape[0], historical_window, 
                                      X_train_tensors.shape[2]))

X_test_tensors_final = torch.reshape(X_test_tensors, 
                                     (X_test_tensors.shape[0], historical_window, 
                                      X_test_tensors.shape[2]))

print("Training shape:", X_train_tensors.shape, y_train_tensors.shape)
print("Testing shape:", X_test_tensors_final.shape, y_test_tensors.shape)

Let's make sure that the data logic of the test set checks out. Sequential data is hard to get our heads around and especially when generating a test set for multistep output models. Here, we want to take the 100 previous predictors up to the current time step and predict 50 time steps into the future. In the test set, we have 150 batch feature samples, each consisting of 100 time steps and 4 feature predictors. 

In the targets for the test set, we again have 150 batch samples, each consisting of an array of length 50 of scalar outputs. 

Since we want a way to validate our results, we need to predict the Bitcoin price for 50 time steps in the test set for which we have the data (i.e. the test targets). Because of the way we wrote the <code> split_sequence()</code>, we just need the last sample of 100 days in <code>X_test</code>, run the model on it and compare those predictions with the last sample of the 50 days of <code>y_test</code>. These correspond to a period of 100 days in the <code>X_test</code>'s last sample, proceeded immediately by the next 50 days in the last sample of <code>y_test</code>. 

In [ ]:
X_check, y_check = split_sequences(X, y.values.reshape(-1,1), historical_window, forward_window) # see above - just doing y.reshape is deprecated
X_check[-1][0:4]

In [ ]:
X.iloc[-149:-145]

The first four rows of the data match. Note that <code>X_check[-1]</code> should be identical to the <code>X.iloc[-149:-49]</code>, ending 50 days before the end of our dataset. Therefore, we are taking the 100 timesteps of information up to the 26th of Nov 2021, and attempting to predict the 50 days after that up to 14th January 2022. Our final check is to make sure that the final batch sample in our test targets matches these data predictions. 

In [ ]:
y_check[-1]

In [ ]:
df["Clock Bias (seconds)"].values[-50:]

Summary: The main performance test for our model will be on the last batch sample in the test set. This will be predictors rom the 100 time steps up to the 26th of Nov 2021 and this will be used by our model to predict the next 50 days of Bitcoin prices up to the 14th of Jan 2022. In this way, we validate model performance by comparing predcitions to the actual prices in that 50 day window. 

# Constructing the LSTM model

Now, let's construct the LSTM class, inheriting from the nn.Module. In constract to the previous univariate LSTM, we're going to build the model with the nn.LSTM rather than nn.LSTMCell because it's good to understand both options and we dont really need the nn.LSTMCell flexibility right now. We know that nn.LSTM is a recurrent application of the nn.LSTMCell. 

In [ ]:
class LSTM(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers): 
        super().__init__()
        self.num_classes = num_classes # output size 
        self.num_layers = num_layers # number of recurrent layers in the LSTM 
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm later

        # LSTM model 
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.fc1 = nn.Linear(hidden_size, 128) # fully connected 
        self.fc2 = nn.Linear(128, num_classes) # last FC
        self.relu = nn.ReLU()

    def forward(self, x): 
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # propgagate input through LSTM
        output, (hn, cn) =self.lstm(x, (h_0, c_0)) # (input, hidden and internal state)
        hn = hn.view(-1, self.hidden_size) # reshape data for Dense layer 
        out = self.relu(hn)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In the initializations, we start with the parent class, nn.Module. Most initializations in PyTorch are separated into two chunks: 

- Any wariables that the class will need for reference, for things such as the hidden layer size, input size, and number of layers
- Defining the layers of the model without connecting them using the variables instatiated above 

Frist we need to shape the LSTM with input size, hidden size and number of recurrent layers. For instance num_layers=2 means stacking 2 LSTMs together to form a stacked LSTM with the second LSTM taking in outputs of the first LSTM and computing the final results. Thus we initialize these three variables in the first part of the __init__. We also need to force our model to output only 1 predicted value so we initialize the number of classes variable. 

In the second part of the __init__, we set out layers of our network with the first later obviously a recurrent aapplication of the LSTM cells, with the parameters specified above: 

<code> self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first= True)</code>

Next, we pass this to a fully connected layer which has an input of the hidden size (the size of the output from the last STM later), and outputs 128 activations. Then we pass these 128 activations to another hidden layer which accepts 128 inputs, and which we want our ouput num_classes (which in this clase would be 1). Finally we pass this activation through a non-linear function, the ReLU. 

<code>self.fc1 = nn.Linear(hidden_size, 128)</code>


<code>self.fc2 = nn.Linear(128, num_classes)</code>


<code>self.relu = nn.ReLU </code>

# Training

Here we will use a typical regression loss function like the MSE.

In [ ]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train, X_test, y_test):
    for epoch in range(n_epochs): 
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient by manually setting to 0 

        # obtain loss function 
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function 
        optimiser.step() # improve the loss, i.e. backprop 

        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)

        if epoch % 2 == 0: 
            print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, loss.item(), test_loss.item()))

Here, we are doing the forward through the network by passing the training tensors, which we shaped before. We then zero out the current gradients in the torch computational graph. We compare these outputs from the forward pass with the actual train targets using our loss function, and backpropagate with loss.backward() to calculate the gradients of the loss wrt to the parameters (the weights and biases). We then use this loss to update the parameters. 

In [ ]:
warnings.filterwarnings('ignore')

n_epochs = 100
learning_rate = 0.001 

input_size = X_ss.shape[2] # num of features
hidden_size = 2 # num of features in the hidden state
num_layers = 1
num_classes = forward_window # num of output classes

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimiser = torch.optim.Adam(lstm.parameters(), lr = learning_rate)

In [ ]:
training_loop(n_epochs=n_epochs, 
              lstm=lstm, 
              optimiser=optimiser, 
              loss_fn = loss_fn, 
              X_train=X_train_tensors_final, 
              y_train=y_train_tensors, 
              X_test=X_test_tensors_final, 
              y_test=y_test_tensors)


# Prediction

A key part of prediction, if the variables have been standardized, is known was we need to pass to the model. That is what do we need to standardize before passing it to hte model. We just apply the same transforms to the predictors and targets so that whatever we feed to the model is exactly what the model is seeing. It wouldnt make sense to not stardize our inpits; the model would be confused as to why they no longer have zero mean and unit variance. 

Let's plot the result for the full dataset

In [ ]:
df_X_ss = ss.transform(df.drop(columns=['File'])) # old transformers 
df_y_mm = mm.transform(df["Clock Bias (seconds)"].values.reshape(-1,1)) # older transformers

#split sequences 
df_X_ss, df_y_mm = split_sequences(df_X_ss, df_y_mm, historical_window, forward_window)

# convert tensors 
df_X_ss = torch.Tensor(df_X_ss)
df_y_mm = torch.Tensor(df_y_mm)

#reshape dataset
df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], historical_window, df_X_ss.shape[2]))

train_predict = lstm(df_X_ss)
data_predict = train_predict.data.numpy()
dataY_plot = df_y_mm.data.numpy()

data_predict = mm.inverse_transform(data_predict)
dataY_plot = mm.inverse_transform(dataY_plot)

true, preds = [], []

for i in range(len(dataY_plot)): 
    true.append(dataY_plot[i][0])
for i in range(len(data_predict)): 
    preds.append(data_predict[i][0])

plt.figure(figsize=(10,6))
test_cutoff = round(0.90 * total_samples)
plt.axvline(x=test_cutoff, c='r', linestyle='--') # size of the training set

plt.plot(true, label='Actual Data')
plt.plot(preds, label='Predicted Data')
plt.title('Time Series Prediction')
plt.legend()
plt.savefig("whole_plot.png", dpi = 300)
plt.show()

In [ ]:
r2 = r2_score(true[test_cutoff:], preds[test_cutoff:])
r2

In [ ]:
mean_absolute_percentage_error(true, preds)

In [ ]:
root_mean_squared_error(true, preds)

Although these results may seem good, it's not actually as good as it appears. This plot above, where the test data is any time step beyond the red-dashed line, seems to suggest that our moel is amazingly accurate at predicting the price of Bitcoin, a few months into the future. However, what's happening here is a form of data leakage, where information about the test targets has leaked into the test features. That is, whenever we run the model, it has access to the <code>Open</code> price for that day, which is obviously going to be extremely close to the price Bitcoin endsup at for the particular time step. When we loop over our predictions on the model, we append the first predicted value for that time-step. This means that every prediction we are plotting on here has the benefit of being the next predicted value - and not the sequence of 50 values into the future (as we set up in our problem definition). 


If we really want to see how our model is performing, we have to feed it the 100 time-steps of features before the final value in the test set and then use one model forward pass to calculate the 50 time-step prices in the test set. 

Recall from above that double-checked our test features in the final batch sample in the test set werre the 100 time steps up to the time-step we're attempting to predict. That is, we feed the model 100 days of information upt to 26th Nov 2021, and get it to predict 50 days of Bitcoin prices, from this day to the 14th Jan 2022. 

In [ ]:
test_predict = lstm(X_test_tensors_final[-1].unsqueeze(0)) # obtains the last sample
test_predict = test_predict.detach().numpy()
test_predict = mm.inverse_transform(test_predict)
test_predict = test_predict[0].tolist()

In [ ]:
test_target = y_test_tensors[-1].detach().numpy()
test_target = mm.inverse_transform(test_target.reshape(1,-1))
test_target = test_target[0].tolist()

In [ ]:
plt.plot(test_target, label = "Actual Data")
plt.plot(test_predict, label = "LSTM Predictions")
plt.savefig("small_plot.png", dpi = 300)
plt.show();

This is good. If we feed the last 100 days of information, our model successfully predicts a steady decline in the price of Bitcoin over the next 50 days. For one last plot, let's put this in perspective of the scale of the data. 

In [ ]:
plt.figure(figsize=(10,6))
a = [x for x in range(2500, len(y))]
plt.plot(a, y[2500:], label="Actual Data"); 
c = [x for x in range (len(y)-forward_window, len(y))]
plt.plot(c, test_predict, label = "One-shot multi-step prediction (50 days)")
plt.axvline(x=len(y)-forward_window, c='r', linestyle="--")
plt.legend()
plt.show()

So, if we'd run our model on 26 Nov 2021, we would have been correct in selling off our Bitcoin. The model correctly predicts a price drop, as well as the rate at which it drops. 

This orange whole curve is generated without looking at ANY target data. Yes, to be fair, the model is very familiar with all the actual targets before the dashed red line. However, once we move past the training set into the test set the model has no idea what the test targets are. It has to generate its prediction  from the input feature only. 

# Conclusion

In [ ]:
r2 = r2_score(test_target, test_predict)
r2

In [ ]:
test_target

In [ ]:
test_predict

In [ ]:

root_mean_squared_error(test_target, test_predict)